In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import NMF
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

print("Bibliothèques chargées !")

Bibliothèques chargées !


In [3]:
# Chemin vers le fichier 
pkl_path = 'vectorisation-du-texte/output/config_L1_S1_LEM1_NG1_FINAL.pkl'

#(Minuscules + Stopwords + Lemmatisation + Unigrammes)

# Chargement du dictionnaire complet
with open(pkl_path, 'rb') as f:
    data = pickle.load(f)

# Extraction des éléments
X = data['X_normalized']       # La matrice numérique (Avis)
y = data['target']             # La cible (Positif/Négatif)
features = data['feature_names'] # La liste des mots (Vocabulaire)
vectorizer = data['tfidf_vectorizer'] # L'outil pour transformer un nouveau texte

print(f"Données chargées : {X.shape[0]} avis et {X.shape[1]} mots de vocabulaire.")

Données chargées : 1848 avis et 1643 mots de vocabulaire.


In [ ]:
# Classification avec régression logistique (Sentiments) :
# 1. Séparation des données (80% pour apprendre, 20% pour tester)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Création du modèle avec GridSearch pour trouver les meilleurs paramètres (L1, L2)
# Le sujet demande de tester différentes régularisations
params = {
    'C': [0.1, 1, 10],            # Force de la régularisation
    'penalty': ['l2'],            # Type de pénalité (l2 est standard et rapide)
    'solver': ['lbfgs']           
}

grid = GridSearchCV(LogisticRegression(max_iter=1000), params, cv=5, n_jobs=-1)
print("Entraînement en cours (patience)...")
grid.fit(X_train, y_train)

# 3. Résultats
best_model_sentiment = grid.best_estimator_
print(f"Meilleur score : {grid.best_score_:.2%}")
print(f"Meilleurs paramètres : {grid.best_params_}")

# Test final
y_pred = best_model_sentiment.predict(X_test)
print("\nRapport de classification :")
print(classification_report(y_test, y_pred))

Entraînement en cours (patience)...
Meilleur score : 88.43%
Meilleurs paramètres : {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}

Rapport de classification :
              precision    recall  f1-score   support

     négatif       0.84      0.93      0.88       188
     positif       0.92      0.81      0.86       182

    accuracy                           0.87       370
   macro avg       0.88      0.87      0.87       370
weighted avg       0.88      0.87      0.87       370



c:\Users\yanni\BUT\BUT3\SAE Data Mining\SAE-5.03-Datamining\venv\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


# ANALYSE DES THEMES AVEC NMF

In [5]:
# On cherche 5 thèmes principaux dans les avis (modifiable avec n_topics)
n_topics = 5
nmf_model = NMF(n_components=n_topics, init='nndsvd', random_state=42)
W = nmf_model.fit_transform(X) # W = Poids des thèmes dans chaque document
H = nmf_model.components_      # H = Poids des mots dans chaque thème

# Fonction pour afficher les mots clés de chaque thème
def display_topics(model, feature_names, no_top_words):
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        topics[topic_idx] = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
        print(f"Thème {topic_idx + 1}: {', '.join(topics[topic_idx])}")
    return topics

print("Thèmes découverts dans les avis :")
topics_found = display_topics(nmf_model, features, 10)

Thèmes découverts dans les avis :
Thème 1: être, ce, le, avoir, je, trop, petit, plus, dommage, tout
Thème 2: recevoir, jamais, avoir, produit, ne, colis, non, article, commande, toujours
Thème 3: boucle, oreille, de, jolie, lui, bel, très, d, bien, photo
Thème 4: qualité, bon, prix, rapport, produit, mauvais, très, bien, super, tre
Thème 5: très, joli, bracelet, beau, bel, bien, cadeau, recommander, bijou, conforme


- Le thème 1 est surement affecter par le bruit (Stopwords mal néttoyer ? (ce, le, je) mais on voit que c'est un thème qui à un rapport avec la taille)

- Problème de livraison 

- Boucle d'oreille (positif largement)

- Qualité/Prix

- Satisfaction / Cadeau ou recommendation

# Cette fonction permets de classifier des nouveaux avis que l'on créer :

In [ ]:
def analyser_nouvel_avis(texte):
    # 1. Nettoyage et Vectorisation (on réutilise le vectorizer sauvegardé !)
    # Attention: le vectorizer attend une liste, donc on met le texte dans []
    # Note: On applique le même prétraitement "brut" que le pipeline si possible, 
    # mais pour l'instant on teste direct.
    vec = vectorizer.transform([texte])
    
    # 2. Prédiction du Sentiment
    sentiment = best_model_sentiment.predict(vec)[0]
    proba = best_model_sentiment.predict_proba(vec).max()
    
    # 3. Prédiction du Thème dominant
    topic_vec = nmf_model.transform(vec)
    topic_id = topic_vec.argmax()
    
    print(f"--- Analyse de : '{texte}' ---")
    print(f"Sentiment : {sentiment.upper()} (Certitude : {proba:.0%})")
    print(f"Sujet principal (Thème {topic_id + 1}) : {', '.join(topics_found[topic_id][:3])}...")
    print("-" * 30)

# Exemples d'analyse de nouveaux avis
analyser_nouvel_avis("Le produit est cassé et la livraison était en retard, nul.")
analyser_nouvel_avis("J'adore ce collier, il brille vraiment bien et le vendeur est sympa.")

--- Analyse de : 'Le produit est cassé et la livraison était en retard, nul.' ---
Sentiment : NÉGATIF (Certitude : 77%)
Sujet principal (Thème 2) : recevoir, jamais, avoir...
------------------------------
--- Analyse de : 'J'adore ce collier, il brille vraiment bien et le vendeur est sympa.' ---
Sentiment : POSITIF (Certitude : 69%)
Sujet principal (Thème 1) : être, ce, le...
------------------------------
